In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
#Loading Dataset
DATASET_PATH = "../Datasets/Covid_Cleaned.csv"
COVID_data = pd.read_csv(DATASET_PATH)

In [5]:
X = COVID_data.drop(columns=['CLASIFFICATION_FINAL'])  # features
y = COVID_data['CLASIFFICATION_FINAL']                 # target (low / high risk of COVID-19)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42) # 70% train, 30% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # test -> 15% validation, 15% final test